In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
plt.style.use('fivethirtyeight')

In [2]:
df = pd.read_csv("daily_data_new.csv", date_parser=True)
df = df.iloc[::-1]

In [3]:
df.head

<bound method NDFrame.head of             Date  Close   Open   High    Low      Vol.  Change %
1183   10/3/2010    0.1    0.1    0.1    0.1   7170.00      0.00
1182   10/4/2010    0.1    0.1    0.1    0.1  34000.00      0.00
1181   10/5/2010    0.1    0.1    0.1    0.1  27530.00      0.00
1180   10/6/2010    0.1    0.1    0.1    0.1  33430.00      0.00
1179   10/7/2010    0.1    0.1    0.1    0.1  43690.00      0.00
...          ...    ...    ...    ...    ...       ...       ...
4     12/25/2013  707.3  702.8  710.0  673.9   5340.00      0.01
3     12/26/2013  802.0  707.3  829.6  707.2  23530.00      0.13
2     12/27/2013  803.0  802.0  836.8  767.9  13890.00      0.00
1     12/28/2013  762.0  803.0  806.0  715.6  13340.00     -0.05
0     12/29/2013  785.0  762.0  804.0  735.0   7040.00      0.03

[1184 rows x 7 columns]>

In [4]:
data_training = df[df['Date']<'2012-01-01'].copy()
data_training

,Date,Close,Open,High,Low,Vol.,Change %
1183,10/3/2010,0.1,0.1,0.1,0.1,7170.00,0.00
1182,10/4/2010,0.1,0.1,0.1,0.1,34000.00,0.00
1181,10/5/2010,0.1,0.1,0.1,0.1,27530.00,0.00
1180,10/6/2010,0.1,0.1,0.1,0.1,33430.00,0.00
1179,10/7/2010,0.1,0.1,0.1,0.1,43690.00,0.00
...,...,...,...,...,...,...,...
4,12/25/2013,707.3,702.8,710.0,673.9,5340.00,0.01
3,12/26/2013,802.0,707.3,829.6,707.2,23530.00,0.13
2,12/27/2013,803.0,802.0,836.8,767.9,13890.00,0.00
1,12/28/2013,762.0,803.0,806.0,715.6,13340.00,-0.05


In [5]:
data_test = df[df['Date']>='2012-01-01'].copy()
data_test

,Date,Close,Open,High,Low,Vol.,Change %
1034,3/1/2011,0.9,0.9,1.0,0.9,22570.00,0.07
1033,3/2/2011,0.9,0.9,0.9,0.9,2740.00,0.00
1032,3/3/2011,0.9,0.9,0.9,0.9,2750.00,0.00
1031,3/4/2011,0.9,0.9,0.9,0.9,2710.00,0.00
1030,3/5/2011,0.9,0.9,0.9,0.8,12570.00,0.00
...,...,...,...,...,...,...,...
94,9/26/2013,137.1,135.0,139.0,134.7,6540.00,0.02
93,9/27/2013,138.9,137.1,142.7,134.8,27140.00,0.01
92,9/28/2013,142.5,138.9,143.0,138.0,13660.00,0.03
91,9/29/2013,143.9,142.5,145.8,141.4,18100.00,0.01


In [6]:
training_data = data_training.drop(['Date', 'Change %'], axis=1)
training_data

,Close,Open,High,Low,Vol.
1183,0.1,0.1,0.1,0.1,7170.00
1182,0.1,0.1,0.1,0.1,34000.00
1181,0.1,0.1,0.1,0.1,27530.00
1180,0.1,0.1,0.1,0.1,33430.00
1179,0.1,0.1,0.1,0.1,43690.00
...,...,...,...,...,...
4,707.3,702.8,710.0,673.9,5340.00
3,802.0,707.3,829.6,707.2,23530.00
2,803.0,802.0,836.8,767.9,13890.00
1,762.0,803.0,806.0,715.6,13340.00


In [7]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.69414015e-05,
        1.83236480e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.69414015e-05,
        8.85555730e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.69414015e-05,
        7.16192869e-02],
       ...,
       [6.48808081e-01, 6.48000000e-01, 6.73779997e-01, 6.67623022e-01,
        3.59143500e-02],
       [6.15676768e-01, 6.48808081e-01, 6.48977291e-01, 6.22152669e-01,
        3.44746348e-02],
       [6.34262626e-01, 6.15676768e-01, 6.47366726e-01, 6.39019301e-01,
        1.79833517e-02]])

In [8]:
training_data.shape[0]

542

In [9]:
x_train = []
y_train = []

In [10]:
for i in range(30,training_data.shape[0]):
    x_train.append(training_data[i-30:i])
    y_train.append(training_data[i,0])

In [11]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [12]:
x_train.shape

(512, 30, 5)

In [13]:
y_train.shape

(512,)

### Build LSTM

In [14]:
regressior = Sequential()

In [15]:
regressior.add(LSTM(units = 15, activation = 'relu', return_sequences = True, input_shape = (x_train.shape[1],5)))
regressior.add(Dropout(0.1))

regressior.add(LSTM(units = 20, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 25, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.3))

regressior.add(LSTM(units = 40, activation = 'relu'))
regressior.add(Dropout(0.4))

regressior.add(Dense(units= 1))

In [16]:
x_train.shape[1],5

(30, 5)

In [17]:
regressior.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 15)            1260      
                                                                 
 dropout (Dropout)           (None, 30, 15)            0         
                                                                 
 lstm_1 (LSTM)               (None, 30, 20)            2880      
                                                                 
 dropout_1 (Dropout)         (None, 30, 20)            0         
                                                                 
 lstm_2 (LSTM)               (None, 30, 25)            4600      
                                                                 
 dropout_2 (Dropout)         (None, 30, 25)            0         
                                                                 
 lstm_3 (LSTM)               (None, 40)                1

In [18]:
regressior.compile(optimizer='adam', loss='mean_squared_error')

In [19]:
regressior.fit(x_train, y_train, epochs=12, batch_size=32)

Epoch 1/12
16/16 [==============================] - 2s 12ms/step - loss: 0.0369
Epoch 2/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0205
Epoch 3/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0165
Epoch 4/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0132
Epoch 5/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0127
Epoch 6/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0068
Epoch 7/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0100
Epoch 8/12
16/16 [==============================] - 0s 12ms/step - loss: 0.0065
Epoch 9/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0061
Epoch 10/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0054
Epoch 11/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0053
Epoch 12/12
16/16 [==============================] - 0s 13ms/step - loss: 0.0043


### Test Data Set

In [20]:
data_test.head()

,Date,Close,Open,High,Low,Vol.,Change %
1034,3/1/2011,0.9,0.9,1.0,0.9,22570.00,0.07
1033,3/2/2011,0.9,0.9,0.9,0.9,2740.00,0.00
1032,3/3/2011,0.9,0.9,0.9,0.9,2750.00,0.00
1031,3/4/2011,0.9,0.9,0.9,0.9,2710.00,0.00
1030,3/5/2011,0.9,0.9,0.9,0.8,12570.00,0.00


In [21]:
data_training.tail(30)

,Date,Close,Open,High,Low,Vol.,Change %
29,11/30/2013,1205.7,1206.9,1232.9,1150.2,15100.00,0.00
28,12/1/2013,1004.4,1205.7,1216.8,840.3,79920.00,-0.17
27,12/2/2013,1096.6,1004.4,1117.7,975.0,37050.00,0.09
26,12/3/2013,1154.9,1096.6,1185.6,1064.9,19340.00,0.05
25,12/4/2013,1237.6,1154.9,1239.7,1131.3,16300.00,0.07
24,12/5/2013,1106.3,1237.6,1239.9,870.0,58440.00,-0.11
23,12/6/2013,845.0,1106.3,1118.9,800.1,53520.00,-0.24
22,12/7/2013,697.0,845.0,896.0,576.0,97660.00,-0.18
21,12/8/2013,804.0,697.0,829.0,653.0,31210.00,0.15
20,12/9/2013,919.0,804.0,980.0,787.7,27630.00,0.14


In [22]:
past_30_days = data_training.tail(30)

In [23]:
df = past_30_days.append(data_test, ignore_index=True)
df = df.drop(['Date', 'Change %'], axis=1)
df.head()

,Close,Open,High,Low,Vol.
0,1205.7,1206.9,1232.9,1150.2,15100.00
1,1004.4,1205.7,1216.8,840.3,79920.00
2,1096.6,1004.4,1117.7,975.0,37050.00
3,1154.9,1096.6,1185.6,1064.9,19340.00
4,1237.6,1154.9,1239.7,1131.3,16300.00


In [24]:
inputs = scaler.transform(df)


ValueError: could not convert string to float: '#VALUE!'